## Уваходныя параметры

In [1]:
import math

t = 20 # хвілін у экспаненцыяльным размеркаванні
lyambda = 2 # цягніка ў гадзіну
nyu = 60 / t
pi = math.e ** (-1 / t)
ro = 1 - lyambda / 60 * math.e ** (-lyambda / 60)

print("pi = ", pi)
print("ro = ", ro)

pi =  0.951229424500714
ro =  0.9677594633172665


In [2]:
def get_generic(a, R0, m):
    R = []
    generic = []
    for i in range(5000000):
        if len(R) == 0:
            R.append(a * R0 % m)
        else:
            R.append(a * R[i-1] % m)
        generic.append(R[i] / m)
        
    return generic

def update_dict(states, s1, s2, s3):
    state = str(s1) + str(s2) + str(s3)
    
    if state in states:
        states[state] += 1
    else:
        states[state] = 1

pi_0 = get_generic(102191, 131, 203563)
ro_0 = get_generic(102191, 132, 203563)

## Імітацыя сістэмы пры 3 цягніках на гары

In [3]:
s1 = 0
s2 = 0
s3 = 0

tact_number = 1000000

states = {}

block_number = 0
lose_number = 0
made_number = 0
task_number = 0

kol_task_in_queue_1 = []
kol_task_in_queue_2 = []
kol_task_in_system = []
queue_times_1 = []
queue_times_2 = []
task_times = []

kanal = 0

index_1 = 0
index_2 = 0

task = []

task_3 = 0
t_3 = 0
task_2 = 0
t_2 = 0
task_1 = 0
t_1 = 0

task_i = 0

mulct_1 = 0

for i in range(tact_number):
    update_dict(states, s1, s2, s3)
    
    #print (s1, s2, s3, pi_0[index_2], ro_0[index_1])
    
    kol_task_in_system.append(s1 + s2 + s3)
    kol_task_in_queue_1.append(s1)
    kol_task_in_queue_2.append(s2)
    
    if (i+1) % 60 == 0:
        if s1 > 0:
            mulct_1 += 1
    
    if s3 == 1:
        kanal += 1
        task[task_3][2] += 1
        
        if pi_0[index_2] > pi:
            s3 = 0
            made_number += 1
            
            task_times.append(task[task_3][2]) 
            
            task_3 += 1
            
        index_2 += 1          
    
    if s2 > 0:
        for j in range(task_2, task_2+s2):
            task[j][1] += 1
            task[j][2] += 1
        
        if s3 == 0:
            s3 = 1
            s2 -= 1
            
            queue_times_2.append(task[task_2][1])
            
            task_2 += 1
        
    if s1 > 0:        
        for j in range(task_1, task_1+s1):
            task[j][0] += 1
            task[j][2] += 1
        
        if s2 < 2:
            s2 += 1
            s1 -= 1
            
            queue_times_1.append(task[task_1][0])
            
            task_1 += 1
        
    if ro_0[index_1] > ro: 
        task.append([0, 0, 0, i]) 
        
        if s3 == 0:
            s3 = 1
            task_1 = task_i + 1            
            task_2 = task_i + 1
            queue_times_1.append(task[task_i][0])
            queue_times_2.append(task[task_i][1])
        elif s2 < 2:
            s2 += 1
            task_1 = task_i + 1
            queue_times_1.append(task[task_i][0])
        else:
            s1 += 1
                           
        task_i += 1
        
        task_number += 1
        
    index_1 += 1
            
states

{'000': 338057,
 '001': 232923,
 '011': 151939,
 '021': 98904,
 '121': 63009,
 '221': 39618,
 '321': 25056,
 '421': 16371,
 '521': 10156,
 '621': 7028,
 '721': 4458,
 '821': 3181,
 '921': 2132,
 '1021': 2254,
 '1121': 1733,
 '1221': 1173,
 '1321': 565,
 '1421': 438,
 '1521': 247,
 '1621': 368,
 '1721': 191,
 '1821': 110,
 '1921': 82,
 '2021': 7}

### Атрыманыя верагоднасці

In [4]:
for key, value in states.items():
    print (key, value / tact_number)

000 0.338057
001 0.232923
011 0.151939
021 0.098904
121 0.063009
221 0.039618
321 0.025056
421 0.016371
521 0.010156
621 0.007028
721 0.004458
821 0.003181
921 0.002132
1021 0.002254
1121 0.001733
1221 0.001173
1321 0.000565
1421 0.000438
1521 0.000247
1621 0.000368
1721 0.000191
1821 0.00011
1921 8.2e-05
2021 7e-06


### Параметры сістэмы

In [5]:
print("P_отк*   ", lose_number / task_number)
print("P_бл*    ", block_number / tact_number)
print("A*       ", made_number / tact_number)
print("Q*       ", made_number / task_number)
print("L_оч_1*  ", sum(kol_task_in_queue_1) / len(kol_task_in_queue_1))
print("L_оч_2*  ", sum(kol_task_in_queue_2) / len(kol_task_in_queue_2))
print("L_c*     ", sum(kol_task_in_system) / len(kol_task_in_system))
print("W_оч_1*  ", sum(queue_times_1) / len(queue_times_1))
print("W_оч_2*  ", sum(queue_times_2) / len(queue_times_2))
print("W_c*     ", sum(task_times) / len(task_times))
print("Ккан*    ", kanal / tact_number)

P_отк*    0.0
P_бл*     0.0
A*        0.032232
Q*        0.9999069334574221
L_оч_1*   0.537361
L_оч_2*   0.706101
L_c*      1.905405
W_оч_1*   16.67011012874205
W_оч_2*   21.90590388732045
W_c*      59.11460660213452
Ккан*     0.661943


### Падлік сумарнай колькасці штрафу

In [6]:
kol = 0

for i in range(len(queue_times_1)):
    kol += (queue_times_1[i] % 60)
    
kol*1000

212101000

## Імітацыя сістэмы пры 4 цягніках на гары

In [7]:
s1 = 0
s2 = 0
s3 = 0

tact_number = 1000000

states = {}

block_number = 0
lose_number = 0
made_number = 0
task_number = 0

kol_task_in_queue_1 = []
kol_task_in_queue_2 = []
kol_task_in_system = []
queue_times_1 = []
queue_times_2 = []
task_times = []

kanal = 0

index_1 = 0
index_2 = 0

task_4 = []

task_3 = 0
t_3 = 0
task_2 = 0
t_2 = 0
task_1 = 0
t_1 = 0

task_i = 0

mulct_2 = 0

for i in range(tact_number):
    update_dict(states, s1, s2, s3)
    
    #print (s1, s2, s3, pi_0[index_2], ro_0[index_1])
    
    kol_task_in_system.append(s1 + s2 + s3)
    kol_task_in_queue_1.append(s1)
    kol_task_in_queue_2.append(s2)
    
    if (i+1) % 60 == 0:
        if s1 > 0:
            mulct_2 += 1
    
    if s3 == 1:
        kanal += 1
        task_4[task_3][2] += 1
        
        if pi_0[index_2] > pi:
            s3 = 0
            made_number += 1
            
            task_times.append(task_4[task_3][2]) 
            
            task_3 += 1
            
        index_2 += 1          
    
    if s2 > 0:
        for j in range(task_2, task_2+s2):
            task_4[j][1] += 1
            task_4[j][2] += 1
        
        if s3 == 0:
            s3 = 1
            s2 -= 1
            
            queue_times_2.append(task_4[task_2][1])
            
            task_2 += 1
        
    if s1 > 0:        
        for j in range(task_1, task_1+s1):
            task_4[j][0] += 1
            task_4[j][2] += 1
        
        if s2 < 3:
            s2 += 1
            s1 -= 1
            
            queue_times_1.append(task_4[task_1][0])
            
            task_1 += 1
        
    if ro_0[index_1] > ro: 
        task_4.append([0, 0, 0, i]) 
        
        if s3 == 0:
            s3 = 1
            task_1 = task_i + 1            
            task_2 = task_i + 1
            queue_times_1.append(task_4[task_i][0])
            queue_times_2.append(task_4[task_i][1])
        elif s2 < 3:
            s2 += 1
            task_1 = task_i + 1  
            queue_times_1.append(task_4[task_i][0])
        else:
            s1 += 1
                           
        task_i += 1
        
        task_number += 1
        
    index_1 += 1
            
states

{'000': 338057,
 '001': 232923,
 '011': 151939,
 '021': 98904,
 '031': 63009,
 '131': 39618,
 '231': 25056,
 '331': 16371,
 '431': 10156,
 '531': 7028,
 '631': 4458,
 '731': 3181,
 '831': 2132,
 '931': 2254,
 '1031': 1733,
 '1131': 1173,
 '1231': 565,
 '1331': 438,
 '1431': 247,
 '1531': 368,
 '1631': 191,
 '1731': 110,
 '1831': 82,
 '1931': 7}

### Атрыманыя верагоднасці

In [8]:
for key, value in states.items():
    print (key, value / tact_number)

000 0.338057
001 0.232923
011 0.151939
021 0.098904
031 0.063009
131 0.039618
231 0.025056
331 0.016371
431 0.010156
531 0.007028
631 0.004458
731 0.003181
831 0.002132
931 0.002254
1031 0.001733
1131 0.001173
1231 0.000565
1331 0.000438
1431 0.000247
1531 0.000368
1631 0.000191
1731 0.00011
1831 8.2e-05
1931 7e-06


### Параметры сістэмы

In [9]:
print("P_отк*   ", lose_number / task_number)
print("P_бл*    ", block_number / tact_number)
print("A*       ", made_number / tact_number, made_number / tact_number * 60)
print("Q*       ", made_number / task_number)
print("L_оч_1*  ", sum(kol_task_in_queue_1) / len(kol_task_in_queue_1))
print("L_оч_2*  ", sum(kol_task_in_queue_2) / len(kol_task_in_queue_2))
print("L_оч*    ", sum(kol_task_in_queue_2) / len(kol_task_in_queue_2) + sum(kol_task_in_queue_1) / len(kol_task_in_queue_1))
print("L_c*     ", sum(kol_task_in_system) / len(kol_task_in_system))
print("W_оч_1*  ", sum(queue_times_1) / len(queue_times_1))
print("W_оч_2*  ", sum(queue_times_2) / len(queue_times_2))
print("W_оч*    ", sum(queue_times_2) / len(queue_times_2) + sum(queue_times_1) / len(queue_times_1))
print("W_c*     ", sum(task_times) / len(task_times))
print("Ккан*    ", kanal / tact_number)

P_отк*    0.0
P_бл*     0.0
A*        0.032232 1.9339199999999999
Q*        0.9999069334574221
L_оч_1*   0.359184
L_оч_2*   0.884278
L_оч*     1.243462
L_c*      1.905405
W_оч_1*   11.142671009771988
W_оч_2*   27.4336859740018
W_оч*     38.57635698377379
W_c*      59.11460660213452
Ккан*     0.661943


### Падлік сумарнай колькасці штрафу

In [10]:
kol = 0

for i in range(len(queue_times_1)):
    kol += (queue_times_1[i] % 60)
    
kol*1000

138384000

## Падлік часу, калі акупіцца прыбаўка адного месца на гару

In [11]:
kol_1 = 0
kol_2 = 0

for i in range(len(task)):
    kol_1 += (task[i][0] % 60)
    kol_2 += (task_4[i][0] % 60)
    
    if (kol_1 - kol_2) * 1000 >= 1000000:
        print(task[i][3] + task_4[i][0])
        print((task[i][3] + task_4[i][0]) / 60 / 24)
        break

14686
10.198611111111111


In [12]:
1000000 /(0.537361-0.359184)/1000/24

233.84986090610278